In [36]:
with open('data/advent16.txt', 'r') as f:
    data = f.read().splitlines()

ranges = data[0:20]
my_ticket = data[22]
tickets = data[25:]

# Parse the ranges
for ix, r in enumerate(ranges):
    line = r.split(' ')
    if ix > 9:
        ranges[ix] = [int(i) for i in line[1].split('-') + line[3].split('-')]
    else:
        ranges[ix] = [int(i) for i in line[2].split('-') + line[4].split('-')]

tickets = [[int(i) for i in line.split(',')] for line in tickets]
my_ticket = [int(i) for i in my_ticket.split(',')]

def in_any_range(num, ranges):
    for r in ranges:
        if (r[0] <= num <= r[1]) or (r[2] <= num <= r[3]):
            return True
    return False

In [37]:
error_rate = 0
invalid_tickets = set([])
for ix, ticket in enumerate(tickets):
    for i in ticket:
        if not in_any_range(i, ranges):
            error_rate += i
            invalid_tickets.add(ix)

print(error_rate)

# Remove invalid tickets (not idempotent!)
invalid_tickets = sorted(list(invalid_tickets), reverse=True)
for ix in invalid_tickets:
    tickets.pop(ix)

24110


In [38]:
def fits_all_nums(nums, r):
    # Does a list of numbers fit this range
    for num in nums:
        if not ((r[0] <= num <= r[1]) or (r[2] <= num <= r[3])):
            return False
    return True

In [ ]:
# Dictionary from ranges to columns
# e.g. {1: 4} means that my_ticket[4] refers to ranges[1]
# Remember, for the excercise we need to identify ranges 0, 1, 2, 3, 4, 5
identified = {}
num_cols = len(my_ticket)

# This doesn't work... it loops infinitely... :(

while not all([i in identified for i in range(6)]):
    # Pass over each column to see if there is only a single range it could be
    unidentified_cols = [i for i in range(num_cols) if i not in identified.values()]
    for col_ix in unidentified_cols:
        nums = [ticket[col_ix] for ticket in tickets]
        fits = [fits_all_nums(nums, r) for r in ranges]
        if sum(fits) == 1:
            print("Found one!", col_ix)
            identified[fits.index(True)] = col_ix
        elif sum(fits) == 0:
            raise ValueError("Something went wrong")


In [57]:
import numpy as np
fits = np.zeros((len(my_ticket), len(ranges)), dtype=bool)

for x in range(len(my_ticket)):
    for y in range(len(ranges)):
        col = [ticket[x] for ticket in tickets]
        fits[x, y] = fits_all_nums(col, ranges[y])

# Look, don't ask too many questions... It just turns out that
# The transpose of the inverse (and ignore the -1 elements) is the answer
solution = np.where(np.transpose(np.linalg.inv(fits) == 1))
fits[solution]


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [63]:
result = 1  # Multiply the ticket values where rangge is in 0 to 5
# first value is column, second value is range
for i, j in zip(solution[0], solution[1]):
    if j < 6:
        result *= my_ticket[i]
result

6766503490793